DRESS COLOR CLASSIFICATION
---------------------------

Connect the google drive to google colab

In [ ]:

from google.colab import drive
drive.mount('/content/drive')



Import library and import dataset from drive 

In [ ]:
import tensorflow as tf


data_dir = '/content/drive/MyDrive/Colab Notebooks/dress_dataset'

for root, dirs, files in os.walk(data_dir):
    print("Root:", root)
    print("Dirs:", dirs)
    print("Files:", files)


Check what is inside the data set 

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True
)
class_names = train_dataset.class_names
print("Class Names:", class_names)


Show sample data in the dataset 

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):  
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()


MAIN PART
--------- 

Import all needed library

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from google.colab import drive


Mount the drive 

In [ ]:

drive.mount('/content/drive')


data_dir = '/content/drive/MyDrive/Colab Notebooks/dress_dataset'


Data train and valitation 

In [ ]:

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    subset="training",
    validation_split=0.2,  
    seed=123
)


val_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    subset="validation",
    validation_split=0.2,
    seed=123
)


class_names = train_dataset.class_names
print("Class Names:", class_names)


Normalize the datasets

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


Create an Simple CNN model 

In [ ]:

model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),  # Input shape
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Output layer
])


Compile the model adn train with epoch

In [ ]:

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


model.fit(train_dataset, validation_data=val_dataset, epochs=3)
